In [1]:
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
import sqlite3
import DataFunctions


configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Data Preprocessing

Games DF

In [2]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"

# no longer need, right?
# try:
#     api_response = api_instance.get_games(year=year,division=division)
#     print(len(api_response))
# except:
#     print(ApiException)
    

In [3]:
gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=2022)
len(gamelist)

735

In [4]:
games_df = DataFunctions.df_from_games(gamelist=gamelist)
games_df.head()

,away_conference,away_id,away_points,away_team,home_conference,home_id,home_points,home_team,id,neutral_site,season
0,Big Ten,158,28,Nebraska,Big Ten,77,31,Northwestern,401405059,True,2022
1,Mountain West,2751,6,Wyoming,Big Ten,356,38,Illinois,401405058,False,2022
2,FBS Independents,41,20,Connecticut,Mountain West,328,31,Utah State,401404146,False,2022
3,Conference USA,2429,13,Charlotte,Conference USA,2226,43,Florida Atlantic,401426530,False,2022
4,Conference USA,249,31,North Texas,Conference USA,2638,13,UTEP,401426531,False,2022


In [5]:
conn = sqlite3.connect("CollegeFootball.db")
games_df.to_sql("games",conn,if_exists="replace",index=False)
conn.close()


In [7]:
# we can see with this code that we can extract DF from SQL db

# conn = sqlite3.connect("CollegeFootball.db")
# trialdf = pd.read_sql_query("SELECT * FROM games",conn)
# conn.close()
# print(trialdf.shape)
# trialdf.head()


Betting lines DF

In [8]:
api_instance1 = cfbd.BettingApi(cfbd.ApiClient(configuration))
api_response1 = api_instance1.get_lines(year=year)

In [9]:
api_response1[1]

{'away_conference': 'Great American',
 'away_score': 7,
 'away_team': 'Northwestern Oklahoma State',
 'home_conference': 'Great American',
 'home_score': 28,
 'home_team': 'Harding',
 'id': 401431455,
 'lines': [],
 'season': 2022,
 'season_type': 'regular',
 'start_date': '2022-10-29T20:00:00.000Z',
 'week': 9}

In [10]:
conferences=[]
for game in gamelist:
    conferences.append(game.away_conference)
conferences=set(conferences)

conferences

{'ACC',
 'American Athletic',
 'Big 12',
 'Big Ten',
 'Conference USA',
 'FBS Independents',
 'Mid-American',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt'}

In [12]:
betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=2022,conferences=conferences)

In [14]:
len(betting_list)

738

Team stats per season DF

Merging tables to set up for NN